In [1]:
!nvidia-smi


Sun Jan 25 16:30:03 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [25]:
%%writefile task2_scan.cu

#include <iostream>              // Ввод и вывод в консоль
#include <vector>                // Контейнер std::vector
#include <chrono>                // Измерение времени выполнения на CPU
#include <cuda_runtime.h>        // CUDA Runtime API

// Размер входного массива
#define N 1000000

// Количество потоков в блоке CUDA
#define THREADS 256

// Макрос для проверки ошибок CUDA
// При возникновении ошибки выводит сообщение и завершает программу
#define CUDA_CHECK(err) \
    if (err != cudaSuccess) { \
        std::cerr << "CUDA error: " << cudaGetErrorString(err) << std::endl; \
        exit(1); \
    }

// Последовательная inclusive scan (префиксная сумма) на CPU
// out[i] = in[0] + in[1] + ... + in[i]
void cpuScan(const std::vector<int>& in, std::vector<int>& out) {
    out[0] = in[0];
    for (int i = 1; i < (int)in.size(); i++) {
        out[i] = out[i - 1] + in[i];
    }
}

// CUDA-ядро: inclusive scan внутри одного блока
// Используется алгоритм Hillis-Steele
// Каждый блок обрабатывает 2 * THREADS элементов
__global__ void blockScanInclusive(int* data, int* blockSums, int n) {

    // Shared memory для хранения данных блока
    // Размер равен количеству элементов, обрабатываемых блоком
    __shared__ int temp[THREADS * 2];

    // Индекс потока внутри блока
    int tid = threadIdx.x;

    // Начальный индекс данных, обрабатываемых блоком
    int start = 2 * blockIdx.x * blockDim.x;

    // Загрузка данных из глобальной памяти в shared memory
    // Каждый поток загружает два элемента
    temp[tid] =
        (start + tid < n) ? data[start + tid] : 0;
    temp[tid + blockDim.x] =
        (start + blockDim.x + tid < n) ? data[start + blockDim.x + tid] : 0;

    // Inclusive scan по алгоритму Hillis-Steele
    // На каждом шаге offset удваивается
    for (int offset = 1; offset < 2 * blockDim.x; offset <<= 1) {
        __syncthreads();

        int val = temp[tid];
        int val2 = temp[tid + blockDim.x];

        // Добавление значения с предыдущего шага
        if (tid >= offset) {
            val += temp[tid - offset];
        }
        if (tid + blockDim.x >= offset) {
            val2 += temp[tid + blockDim.x - offset];
        }

        __syncthreads();

        // Запись обновлённых значений
        temp[tid] = val;
        temp[tid + blockDim.x] = val2;
    }

    __syncthreads();

    // Первый поток блока сохраняет сумму всех элементов блока
    if (tid == 0) {
        blockSums[blockIdx.x] = temp[2 * blockDim.x - 1];
    }

    // Запись результатов сканирования обратно в глобальную память
    if (start + tid < n) {
        data[start + tid] = temp[tid];
    }
    if (start + blockDim.x + tid < n) {
        data[start + blockDim.x + tid] = temp[tid + blockDim.x];
    }
}

// CUDA-ядро для добавления префиксной суммы блока
// к каждому элементу соответствующего блока
__global__ void addBlockPrefix(int* data, int* blockPrefix, int n) {

    // Индекс потока
    int tid = threadIdx.x;

    // Начальный индекс блока
    int start = 2 * blockIdx.x * blockDim.x;

    // Префиксная сумма предыдущих блоков
    int prefix = blockPrefix[blockIdx.x];

    // Добавление префикса к элементам блока
    if (start + tid < n) {
        data[start + tid] += prefix;
    }
    if (start + blockDim.x + tid < n) {
        data[start + blockDim.x + tid] += prefix;
    }
}

int main() {

    // Входной массив на CPU, заполненный единицами
    std::vector<int> h_in(N, 1);

    // Результат CPU-сканирования
    std::vector<int> h_out(N);

    // ---------------- CPU ----------------

    // Засекаем время начала вычислений на CPU
    auto cpu_start = std::chrono::high_resolution_clock::now();

    // Выполнение последовательного scan на CPU
    cpuScan(h_in, h_out);

    // Засекаем время окончания
    auto cpu_end = std::chrono::high_resolution_clock::now();

    // Вычисляем время выполнения в миллисекундах
    std::chrono::duration<double, std::milli> cpu_time =
        cpu_end - cpu_start;

    // ---------------- GPU ----------------

    // Указатели на память GPU
    int* d_data;
    int* d_blockSums;

    // Выделение памяти под входные данные на GPU
    CUDA_CHECK(cudaMalloc(&d_data, N * sizeof(int)));

    // Количество блоков CUDA
    int blocks =
        (N + (THREADS * 2 - 1)) / (THREADS * 2);

    // Выделение памяти под суммы блоков
    CUDA_CHECK(cudaMalloc(&d_blockSums,
                           blocks * sizeof(int)));

    // Копирование входных данных с CPU на GPU
    CUDA_CHECK(cudaMemcpy(d_data,
                           h_in.data(),
                           N * sizeof(int),
                           cudaMemcpyHostToDevice));

    // CUDA-события для измерения времени GPU
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);

    // 1) Inclusive scan внутри каждого блока на GPU
    blockScanInclusive<<<blocks, THREADS>>>(
        d_data, d_blockSums, N);
    CUDA_CHECK(cudaDeviceSynchronize());

    // 2) Exclusive scan сумм блоков на CPU
    std::vector<int> h_blockSums(blocks);
    CUDA_CHECK(cudaMemcpy(h_blockSums.data(),
                           d_blockSums,
                           blocks * sizeof(int),
                           cudaMemcpyDeviceToHost));

    // Вычисление префиксных сумм блоков
    std::vector<int> h_blockPrefix(blocks);
    h_blockPrefix[0] = 0;
    for (int i = 1; i < blocks; i++) {
        h_blockPrefix[i] =
            h_blockPrefix[i - 1] + h_blockSums[i - 1];
    }

    // Копирование префиксных сумм блоков обратно на GPU
    CUDA_CHECK(cudaMemcpy(d_blockSums,
                           h_blockPrefix.data(),
                           blocks * sizeof(int),
                           cudaMemcpyHostToDevice));

    // 3) Добавление префиксной суммы блока
    addBlockPrefix<<<blocks, THREADS>>>(
        d_data, d_blockSums, N);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Вычисление времени выполнения GPU
    float gpu_time = 0;
    cudaEventElapsedTime(&gpu_time, start, stop);

    // Копирование результата GPU обратно на CPU
    std::vector<int> gpu_out(N);
    CUDA_CHECK(cudaMemcpy(gpu_out.data(),
                           d_data,
                           N * sizeof(int),
                           cudaMemcpyDeviceToHost));

    // Проверка корректности результата
    std::cout << "CPU out[0]: " << h_out[0]
              << " GPU out[0]: " << gpu_out[0] << std::endl;
    std::cout << "CPU out[999999]: " << h_out[N - 1]
              << " GPU out[999999]: " << gpu_out[N - 1] << std::endl;

    // Вывод времени выполнения
    std::cout << "CPU time (ms): " << cpu_time.count() << std::endl;
    std::cout << "GPU time (ms): " << gpu_time << std::endl;

    // Освобождение памяти GPU
    cudaFree(d_data);
    cudaFree(d_blockSums);

    return 0;
}



Overwriting task2_scan.cu


In [26]:
!nvcc -arch=sm_75 task2_scan.cu -o task2_scan

In [27]:

!./task2_scan

CPU out[0]: 1 GPU out[0]: 1
CPU out[999999]: 1000000 GPU out[999999]: 1000000
CPU time (ms): 8.52998
GPU time (ms): 0.306976
